目前進度：我在閱讀 SimpleTransformer 的仔入資料的程式碼並嘗試複製<br>
因為我要把它改為多標籤的資料<br>
但我看不太懂，還在努力中

多花一點時間相信我可以掌握，加油

0623 19:00 進度<br>
我把關鍵的部分抓出來了，load_and_cache_examples 就是把檔名轉為 Tensor 的部分！明天繼續

In [2]:
from __future__ import absolute_import, division, print_function

import logging
import math
import os
import random
import tempfile
import warnings
from dataclasses import asdict
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from seqeval.metrics import (
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)
from simpletransformers.config.model_args import NERArgs
from simpletransformers.config.utils import sweep_config_to_sweep_values
from simpletransformers.ner.ner_utils import (
    InputExample,
    LazyNERDataset,
    convert_examples_to_features,
    get_examples_from_df,
    load_hf_dataset,
    read_examples_from_file,
)
from tensorboardX import SummaryWriter
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tqdm.auto import tqdm, trange
from transformers import (
    AlbertConfig,
    AlbertForTokenClassification,
    AlbertTokenizer,
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    BertConfig,
    BertForTokenClassification,
    BertTokenizer,
    BertweetTokenizer,
    BigBirdConfig,
    BigBirdForTokenClassification,
    BigBirdTokenizer,
    CamembertConfig,
    CamembertForTokenClassification,
    CamembertTokenizer,
    DebertaConfig,
    DebertaForTokenClassification,
    DebertaTokenizer,
    DebertaV2Config,
    DebertaV2ForTokenClassification,
    DebertaV2Tokenizer,
    DistilBertConfig,
    DistilBertForTokenClassification,
    DistilBertTokenizer,
    ElectraConfig,
    ElectraForTokenClassification,
    ElectraTokenizer,
    LayoutLMConfig,
    LayoutLMForTokenClassification,
    LayoutLMTokenizer,
    LongformerConfig,
    LongformerForTokenClassification,
    LongformerTokenizer,
    MPNetConfig,
    MPNetForTokenClassification,
    MPNetTokenizer,
    MobileBertConfig,
    MobileBertForTokenClassification,
    MobileBertTokenizer,
    RobertaConfig,
    RobertaForTokenClassification,
    RobertaTokenizerFast,
    SqueezeBertConfig,
    SqueezeBertForTokenClassification,
    SqueezeBertTokenizer,
    XLMConfig,
    XLMForTokenClassification,
    XLMTokenizer,
    XLMRobertaConfig,
    XLMRobertaForTokenClassification,
    XLMRobertaTokenizer,
    XLNetConfig,
    XLNetForTokenClassification,
    XLNetTokenizerFast,
)
from transformers.convert_graph_to_onnx import convert, quantize
from transformers.optimization import AdamW, Adafactor
from transformers.optimization import (
    get_constant_schedule,
    get_constant_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup,
)


In [3]:

from simpletransformers.ner import NERModel
# Create a NERModel
#model = NERModel('bert', 'bert-base-cased')
model = NERModel('bert', 'dslim/bert-base-NER', args={
    'learning_rate': 2e-5,
    'overwrite_output_dir': True,
    'reprocess_input_data': True,
    'num_train_epochs': 1,
    "train_batch_size": 15})

In [4]:
file = "./train.txt"
out = []
with open(file) as f:
    lines = f.readlines()
    
    
for i in range(len(lines)):
    out.append(" ".join(lines[i].replace("\n", "").split(" ")[:-1] + ["O\n"]))

In [5]:
out[:10]

['-DOCSTART- -X- -X- O\n',
 'O\n',
 'EU NNP B-NP O\n',
 'rejects VBZ B-VP O\n',
 'German JJ B-NP O\n',
 'call NN I-NP O\n',
 'to TO B-VP O\n',
 'boycott VB I-VP O\n',
 'British JJ B-NP O\n',
 'lamb NN I-NP O\n']

In [6]:
with open("./test_test.txt", "w") as f:
    f.writelines(out)

In [7]:
lines[:10]

['-DOCSTART- -X- -X- O\n',
 '\n',
 'EU NNP B-NP B-ORG\n',
 'rejects VBZ B-VP O\n',
 'German JJ B-NP B-MISC\n',
 'call NN I-NP O\n',
 'to TO B-VP O\n',
 'boycott VB I-VP O\n',
 'British JJ B-NP B-MISC\n',
 'lamb NN I-NP O\n']

In [8]:
lines[4].replace("\n", "").split(" ")[:-1]

['German', 'JJ', 'B-NP']

In [9]:
" ".join(lines[4].replace("\n", "").split(" ")[:-1] + ["O\n"])

'German JJ B-NP O\n'

In [10]:
out = lines

In [11]:
lines[-2:]

['2 CD I-NP O\n', '\n']

In [12]:
tokenizer = model.tokenizer

In [13]:
tokenizer.encode("Hello")

[101, 8667, 102]

In [14]:
tokenizer.encode("Hello World Eason")

[101, 8667, 1291, 142, 2225, 1320, 102]

In [15]:
tokenizer.encode("World")

[101, 1291, 102]

In [16]:
tokenizer.encode("Eason")

[101, 142, 2225, 1320, 102]

In [21]:
logger = logging.getLogger(__name__)
MODELS_WITH_EXTRA_SEP_TOKEN = [
    "roberta",
    "camembert",
    "xlmroberta",
    "longformer",
    "mpnet",
]

In [22]:
data = "./train.txt"
evaluate=False
no_cache=False
to_predict=None

process_count = model.args.process_count

tokenizer = model.tokenizer
args = model.args

if not no_cache:
    no_cache = args.no_cache

mode = "dev" if evaluate else "train"
if model.args.use_hf_datasets and data is not None:
    if model.args.model_type == "layoutlm":
        raise NotImplementedError(
            "HuggingFace Datasets support is not implemented for LayoutLM models"
        )
    dataset = load_hf_dataset(
        data,
        model.args.labels_list,
        model.args.max_seq_length,
        model.tokenizer,
        # XLNet has a CLS token at the end
        cls_token_at_end=bool(args.model_type in ["xlnet"]),
        cls_token=tokenizer.cls_token_id,
        cls_token_segment_id=2 if args.model_type in ["xlnet"] else 0,
        sep_token=tokenizer.sep_token_id,
        # RoBERTa uses an extra separator b/w pairs of sentences,
        # cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
        sep_token_extra=args.model_type in MODELS_WITH_EXTRA_SEP_TOKEN,
        # PAD on the left for XLNet
        pad_on_left=bool(args.model_type in ["xlnet"]),
        pad_token=tokenizer.pad_token_id,
        pad_token_segment_id=4 if args.model_type in ["xlnet"] else 0,
        pad_token_label_id=model.pad_token_label_id,
        silent=args.silent,
        args=model.args,
    )
else:
    if not to_predict and isinstance(data, str) and model.args.lazy_loading:
        dataset = LazyNERDataset(data, tokenizer, model.args)
    else:
        if to_predict:
            examples = to_predict
            no_cache = True
        else:
            if isinstance(data, str):
                examples = read_examples_from_file(
                    data,
                    mode,
                    bbox=True if model.args.model_type == "layoutlm" else False,
                )
            else:
                if model.args.lazy_loading:
                    raise ValueError(
                        "Input must be given as a path to a file when using lazy loading"
                    )
                examples = get_examples_from_df(
                    data,
                    bbox=True if model.args.model_type == "layoutlm" else False,
                )

        cached_features_file = os.path.join(
            args.cache_dir,
            "cached_{}_{}_{}_{}_{}".format(
                mode,
                args.model_type,
                args.max_seq_length,
                model.num_labels,
                len(examples),
            ),
        )
        if not no_cache:
            os.makedirs(model.args.cache_dir, exist_ok=True)

        if os.path.exists(cached_features_file) and (
            (not args.reprocess_input_data and not no_cache)
            or (
                mode == "dev" and args.use_cached_eval_features and not no_cache
            )
        ):
            features = torch.load(cached_features_file)
            logger.info(
                f" Features loaded from cache at {cached_features_file}"
            )
        else:
            logger.info(" Converting to features started.")
            features = convert_examples_to_features(
                examples,
                model.args.labels_list,
                model.args.max_seq_length,
                model.tokenizer,
                # XLNet has a CLS token at the end
                cls_token_at_end=bool(args.model_type in ["xlnet"]),
                cls_token=tokenizer.cls_token,
                cls_token_segment_id=2 if args.model_type in ["xlnet"] else 0,
                sep_token=tokenizer.sep_token,
                # RoBERTa uses an extra separator b/w pairs of sentences,
                # cf. github.com/pytorch/fairseq/commit/1684e166e3da03f5b600dbb7855cb98ddfcd0805
                sep_token_extra=args.model_type in MODELS_WITH_EXTRA_SEP_TOKEN,
                # PAD on the left for XLNet
                pad_on_left=bool(args.model_type in ["xlnet"]),
                pad_token=tokenizer.pad_token_id,
                pad_token_segment_id=4 if args.model_type in ["xlnet"] else 0,
                pad_token_label_id=model.pad_token_label_id,
                process_count=process_count,
                silent=args.silent,
                use_multiprocessing=args.use_multiprocessing,
                chunksize=args.multiprocessing_chunksize,
                mode=mode,
                use_multiprocessing_for_evaluation=args.use_multiprocessing_for_evaluation,
            )

            if not no_cache:
                torch.save(features, cached_features_file)

        all_input_ids = torch.tensor(
            [f.input_ids for f in features], dtype=torch.long
        )
        all_input_mask = torch.tensor(
            [f.input_mask for f in features], dtype=torch.long
        )
        all_segment_ids = torch.tensor(
            [f.segment_ids for f in features], dtype=torch.long
        )
        all_label_ids = torch.tensor(
            [f.label_ids for f in features], dtype=torch.long
        )

        if model.args.model_type == "layoutlm":
            all_bboxes = torch.tensor(
                [f.bboxes for f in features], dtype=torch.long
            )

        if model.args.onnx:
            out_return = all_label_ids

        if model.args.model_type == "layoutlm":
            dataset = TensorDataset(
                all_input_ids,
                all_input_mask,
                all_segment_ids,
                all_label_ids,
                all_bboxes,
            )
        else:
            dataset = TensorDataset(
                all_input_ids, all_input_mask, all_segment_ids, all_label_ids
            )

out_return =  dataset

In [23]:
out_return